In [1]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
import warnings
warnings.filterwarnings("ignore")
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import confusion_matrix

np.random.seed(123)

In [2]:
# NETWORK = 'InceptionResNetV2'
# NETWORK = 'InceptionV3'
# NETWORK = 'Resnet_FC'
# NETWORK = 'Resnet'
# NETWORK = 'Resnet_res2c'
# NETWORK = 'Resnet_res3d'
# NETWORK = 'Resnet_res4f'
# NETWORK = 'VGG16'
# NETWORK = 'VGG19'
# NETWORK = 'Xception'
# NETWORK = 'DenseNet201'
# NETWORK = 'EfficientNetB1'
# NETWORK = 'EfficientNetB4'
# NETWORK = 'EfficientNetB7'
# NETWORK = 'EfficientNetV2M'
NETWORK = 'NASNetLarge'
# NETWORK = 'ResNet152V2'
# NETWORK = 'ConvNeXtBase'

METHOD = "MAX-ICA"

In [3]:
WORKING_DIRECTORY = '/home/amirhosein/HECKTOR2022/WHOLEIMAGE_MAMIP'

PET_features_filename = f"Features_{NETWORK}_MA-MIP_WHOLE-IMAGE_{METHOD}.csv"

features_path = os.path.join(WORKING_DIRECTORY, f"ExtractedFeatures_{METHOD}")
processed_features_path = os.path.join(WORKING_DIRECTORY, f"ProcessedFeatures_{METHOD}")

processed_features_filename = f"Processed_Features_{NETWORK}_MA-MIP_WHOLE-IMAGE_{METHOD}.csv"
processed_features_full_filename = os.path.join(processed_features_path, processed_features_filename)

PET_featurs_full_path = os.path.join(features_path, PET_features_filename)

PET_filename = PET_features_filename.split(".")[0]

PET_outcome_data = pd.read_csv(PET_featurs_full_path)

outcome_file = "/home/amirhosein/HECKTOR2022/hecktor2022_patient_endpoint_training.csv"

In [4]:
PET_outcome_data.head()

,Unnamed: 0,0,1,2,3,4,5,6,7,8,...,1014,1015,1016,1017,1018,1019,1020,1021,1022,1023
0,CHUM-001,0.005612,-0.006730,-0.033660,-0.039850,-0.015216,0.010274,0.030356,-0.010767,0.027150,...,-0.000236,0.002358,0.001479,-0.011316,0.024758,0.033177,0.015490,-0.049104,-0.020529,-0.024631
1,CHUM-002,0.010851,-0.005613,-0.026061,-0.064302,-0.007999,0.019053,0.044558,-0.026651,0.030483,...,0.000611,0.006583,-0.018491,-0.020962,0.006944,0.035530,0.005182,-0.025923,-0.045458,-0.039574
2,CHUM-006,0.040169,-0.010290,-0.027183,-0.052445,-0.010835,0.023082,0.039642,0.012529,0.017835,...,0.017721,-0.014141,0.003560,-0.021528,0.014641,0.037036,0.022870,0.009854,-0.039887,-0.017344
3,CHUM-007,0.031749,0.005050,-0.028010,-0.075642,0.002801,0.025082,0.043006,-0.004845,-0.001517,...,0.009213,-0.010652,-0.005604,-0.019537,0.029907,0.044327,0.013830,-0.002373,-0.025586,-0.023986
4,CHUM-008,-0.015953,0.008404,0.027935,0.075550,0.020363,-0.007362,-0.036060,-0.016127,-0.011560,...,-0.012799,0.015308,-0.021906,0.019779,-0.024343,-0.026666,-0.025648,-0.012027,-0.001504,0.027299


# Up to here

Read Outcome csv

In [5]:
OD = pd.read_csv(outcome_file)
OD = OD.iloc[:,:]
OD.head()

,PatientID,Relapse,RFS
0,CHUM-001,0,1704
1,CHUM-002,1,439
2,CHUM-006,0,1186
3,CHUM-007,0,1702
4,CHUM-008,0,1499


In [6]:
OD = OD.rename(columns={'PatientID': 'Patient_ID'})
OD.head()

,Patient_ID,Relapse,RFS
0,CHUM-001,0,1704
1,CHUM-002,1,439
2,CHUM-006,0,1186
3,CHUM-007,0,1702
4,CHUM-008,0,1499


In [7]:
OD.columns[0]

'Patient_ID'

In [8]:
first_column = PET_outcome_data.columns[0]
PET_outcome_data = PET_outcome_data.rename(columns={first_column: 'Patient_ID'})

In [9]:
OD.columns[0]


'Patient_ID'

In [10]:
PET_outcome_data.columns[0]

'Patient_ID'

In [11]:
PET_outcome_data.head()

,Patient_ID,0,1,2,3,4,5,6,7,8,...,1014,1015,1016,1017,1018,1019,1020,1021,1022,1023
0,CHUM-001,0.005612,-0.006730,-0.033660,-0.039850,-0.015216,0.010274,0.030356,-0.010767,0.027150,...,-0.000236,0.002358,0.001479,-0.011316,0.024758,0.033177,0.015490,-0.049104,-0.020529,-0.024631
1,CHUM-002,0.010851,-0.005613,-0.026061,-0.064302,-0.007999,0.019053,0.044558,-0.026651,0.030483,...,0.000611,0.006583,-0.018491,-0.020962,0.006944,0.035530,0.005182,-0.025923,-0.045458,-0.039574
2,CHUM-006,0.040169,-0.010290,-0.027183,-0.052445,-0.010835,0.023082,0.039642,0.012529,0.017835,...,0.017721,-0.014141,0.003560,-0.021528,0.014641,0.037036,0.022870,0.009854,-0.039887,-0.017344
3,CHUM-007,0.031749,0.005050,-0.028010,-0.075642,0.002801,0.025082,0.043006,-0.004845,-0.001517,...,0.009213,-0.010652,-0.005604,-0.019537,0.029907,0.044327,0.013830,-0.002373,-0.025586,-0.023986
4,CHUM-008,-0.015953,0.008404,0.027935,0.075550,0.020363,-0.007362,-0.036060,-0.016127,-0.011560,...,-0.012799,0.015308,-0.021906,0.019779,-0.024343,-0.026666,-0.025648,-0.012027,-0.001504,0.027299


In [12]:
# ranges_list = list()
# for col in PET_outcome_data.iloc[:, 1:]:
#     col_range = PET_outcome_data[col].max() - PET_outcome_data[col].min()
#     ranges_list.append(col_range)
#     print(f"Range of column {col}: {col_range}")
# print(f"maximum range: {max(ranges_list)}")

In [13]:
full_features_df = pd.merge(PET_outcome_data, OD, on=OD.columns[0])

In [14]:
full_features_df.head()

,Patient_ID,0,1,2,3,4,5,6,7,8,...,1016,1017,1018,1019,1020,1021,1022,1023,Relapse,RFS
0,CHUM-001,0.005612,-0.006730,-0.033660,-0.039850,-0.015216,0.010274,0.030356,-0.010767,0.027150,...,0.001479,-0.011316,0.024758,0.033177,0.015490,-0.049104,-0.020529,-0.024631,0,1704
1,CHUM-002,0.010851,-0.005613,-0.026061,-0.064302,-0.007999,0.019053,0.044558,-0.026651,0.030483,...,-0.018491,-0.020962,0.006944,0.035530,0.005182,-0.025923,-0.045458,-0.039574,1,439
2,CHUM-006,0.040169,-0.010290,-0.027183,-0.052445,-0.010835,0.023082,0.039642,0.012529,0.017835,...,0.003560,-0.021528,0.014641,0.037036,0.022870,0.009854,-0.039887,-0.017344,0,1186
3,CHUM-007,0.031749,0.005050,-0.028010,-0.075642,0.002801,0.025082,0.043006,-0.004845,-0.001517,...,-0.005604,-0.019537,0.029907,0.044327,0.013830,-0.002373,-0.025586,-0.023986,0,1702
4,CHUM-008,-0.015953,0.008404,0.027935,0.075550,0.020363,-0.007362,-0.036060,-0.016127,-0.011560,...,-0.021906,0.019779,-0.024343,-0.026666,-0.025648,-0.012027,-0.001504,0.027299,0,1499


In [15]:
full_features_df.to_csv(processed_features_full_filename, encoding='utf-8', index=False)